# Imports

In [1]:
import numpy as np

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error , mean_squared_error

import tensorflow as tf
from tensorflow.keras.layers import LSTM , Dropout , Dense
from tensorflow.keras.models import Sequential

import pandas as pd 
import matplotlib.pyplot as plt

import os

In [2]:
filepath = '/home/rahul/Desktop/data/EQ_daily'
not_data = [ 'BANDHANBNK' ,'AXISBANK' ,]
stocks_list = [ 'HDFCBANK' , 'ICICIBANK' , 'KOTAKBANK' , 'SBIN' , 'INDUSINDBK' , 
               'FEDERALBNK' ,'IDFC' , 'RBLBANK' , 'IDEA' ,
                  'GMRINFRA' , 'PNB' , 'BHEL' , 'NATIONALUM' ,'SAIL' ,'BANKBARODA' ,'IOC',
                    'L&TFH' ,'TATAPOWER' , 'NTPC' , 'ONGC' ,'ASHOKLEY' ,'PFC']
csv_files = []
for i in stocks_list:
    csv_files.append(filepath+'/'+ i +'EQ.csv')

In [3]:
dataframe_list = []
ticker_list = []

for i , idx in enumerate(csv_files):
    df = pd.read_csv(idx)
    df['Date'] = pd.to_datetime(df['Date'])
    df.index = df['Date']
    df.drop(df[df['High']==df['Low']].index , axis=0 , inplace=True)
    ticker_list.append(idx.split('.')[0])
    df.drop(['Ticker' , 'Date' , 'Time' , 'Volume' , 'OI '] , axis=1 , inplace=True)
    X = df.copy()
    dataframe_list.append(X)

In [4]:
dataframe_list[1]

,Open,High,Low,Close
Date,,,,
1997-09-24,6.3636,9.1364,6.3636,8.9818
1997-09-25,9.0545,9.5000,8.9091,9.4273
1997-09-26,9.4545,9.5000,9.2273,9.3364
1997-09-29,9.4182,9.4364,9.1091,9.1909
1997-09-30,9.0909,9.1636,8.6818,8.7455
...,...,...,...,...
2019-05-24,414.0000,433.4000,413.1500,432.5000
2019-05-27,432.5000,438.3000,430.0000,435.0500
2019-05-28,435.1500,438.0000,428.5000,437.0000


In [5]:
Training_data = []
Testing_data = []

for i in range(len(dataframe_list)):
    train_size = int(len(dataframe_list[i])* 0.80)
    train = dataframe_list[i][:train_size] 
    test = dataframe_list[i][train_size:]
    Training_data.append(train)
    Testing_data.append(test)

In [6]:
scaled_df = []
scaler_list = []
for i in range(len(dataframe_list)):
    scaler = MinMaxScaler(feature_range=(0,1))
    Close = scaler.fit_transform(np.array(Training_data[i]['Close']).reshape(-1,1))
    Open = scaler.transform(np.array(Training_data[i]['Open']).reshape(-1,1))
    High = scaler.transform(np.array(Training_data[i]['High']).reshape(-1,1))
    Low = scaler.transform(np.array(Training_data[i]['Low']).reshape(-1,1))
    
    open_list = []
    close_list = []
    high_list = []
    low_list = []
    for i in range(len(Open)):
        open_list.append(Open[i])
        high_list.append(High[i])
        low_list.append(Low[i])
        close_list.append(Close[i])
        
    open_arr = np.array(open_list).reshape(-1,1)
    high_arr = np.array(high_list).reshape(-1,1)
    low_arr = np.array(low_list).reshape(-1,1)
    close_arr = np.array(close_list).reshape(-1,1)
    
    scaled_data = pd.DataFrame(open_arr , columns=['Open'])
    scaled_data['High'] = high_arr
    scaled_data['Low'] = low_arr
    scaled_data['Close'] = close_arr
    
    transformed = np.array(scaled_data)
        
    scaler_list.append(scaler)
    scaled_df.append(transformed)

In [7]:
X_train_list = []
y_train_list = []

periods = 60

for i in range(len(Training_data)):
    xappend = []
    yappend = []
    for j in range(periods , len(Training_data[i])-1):
        
        k = np.array(Training_data[i][j-periods :j ]).reshape(-1,240)[0]
        
        xappend.append(k)
        yappend.append(Training_data[i][periods+1:periods+2]['Close'])
        
    X_train_list.append(xappend)
    y_train_list.append(yappend)


In [8]:
X_test_list = []
y_test_list = []

periods = 60

for i in range(len(Testing_data)):
    xappend = []
    yappend = []
    for j in range(periods , len(Testing_data[i])-1):
        k = np.array(Testing_data[i][j-periods:j]).reshape(-1,240)[0]
        
        xappend.append(k)
        yappend.append(Testing_data[i][periods+1:periods+2]['Close'])
        
    X_test_list.append(xappend)
    y_test_list.append(yappend)

In [9]:
base_model = Sequential()
base_model.add(LSTM(240 , return_sequences=True , input_shape=(240,1)))
base_model.add(LSTM(240 , return_sequences=True))
base_model.add(LSTM(240))
base_model.add(Dense(1))
base_model.compile(loss='mean_squared_error' , optimizer='adam' , metrics=['accuracy','mse'])

In [10]:
base_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 240, 240)          232320    
_________________________________________________________________
lstm_1 (LSTM)                (None, 240, 240)          461760    
_________________________________________________________________
lstm_2 (LSTM)                (None, 240)               461760    
_________________________________________________________________
dense (Dense)                (None, 1)                 241       
Total params: 1,156,081
Trainable params: 1,156,081
Non-trainable params: 0
_________________________________________________________________


In [29]:
models_list = []

for i in range(len(X_train_list)):
    X_train = np.array(X_train_list[i]).reshape((-1,240,1))
    y_train = np.array(y_train_list[i]).reshape((-1,1))
    
    model = base_model
    print('############### End Training for {} #####################'.format(ticker_list[i].split('.')[0]))

    model.fit(X_train , y_train)
    models_list.append(model)
    model.save('./models/equity/{}.h5'.format(ticker_list[i].split('.')[0]))
    
    print('############### End Training for {} #####################'.format(ticker_list[i].split('.')[0]))

############### End Training for /home/rahul/Desktop/data/EQ_daily/HDFCBANKEQ #####################


TypeError: 'NoneType' object is not callable